In [1]:
import numpy as np
import pandas as pd

**Cardiovascular Disease**<br>
Consider the Physicians’ Health Study data presented in Example 10.37 (p. 411).

Ex 10.36:<br>
Participants were 22,000 male physicians ages 40−84 and free of cardiovascular disease in 1982. The physicians were randomized to either active aspirin (one white pill containing 325 mg of aspirin taken every other day) or aspirin placebo (one white placebo pill taken every other day). As the study progressed, it was estimated from self-report that 10% of the participants in the aspirin group were not complying (that is, were not taking their study [aspirin] capsules). Thus, the dropout rate was 10%. Also, it was estimated from self-report that 5% of the participants in the placebo group were taking aspirin regularly on their own outside the study protocol. Thus, the drop-in rate was 5%.

Ex 10.37:<br>
Suppose we assume that the incidence of MI is .005 per year among participants who actually take placebo and that aspirin prevents 20% of MIs (i.e., relative risk = p1/p2 = 0.8). We also assume that the duration of the study is 5 years and that the dropout rate in the aspirin group = 10% and the drop- in rate in the placebo group = 5%. How many participants need to be enrolled in each group to achieve 80% power using a two-sided test with significance level = .05?

*The incidence of MI is 0.005 **per year** for the placebo group, therefore the 5-year incidence rate of MI is 5 * 0.005 = 0.025*

10.1 How many participants need to be enrolled in each group to have a 90% chance of detecting a significant difference using a two-sided test with α = .05 if compliance is perfect?

In [2]:
import statsmodels.stats.api as sms
p1 = 0.025 # placebo rate
p2 = 0.02 # p1/p2 = 0.8 # treatment rate
power = 0.9
alpha = 0.05
ratio = 1
nobs1 = None
es = sms.proportion_effectsize(prop1=p1, prop2=p2, method='normal')
n = sms.NormalIndPower().solve_power(es, nobs1, alpha, power, ratio=1, alternative='two-sided')
print(f'Number of subjects per group: {round(n)}')
print(f'Number of subjects in the study: {round(2*n)}')

Number of subjects per group: 18431
Number of subjects in the study: 36863


~18,431 subjects per group (or 36,862 total) are need to be enrolled in each group to have a 90% chance of measuring a 20% drop in relative risk in the case group compared with the control group, at the 5% significance level. 

10.2 Answer Problem 10.1 if compliance is as given in Example 10.37.

λ1 = dropout rate = proportion of participants in the active-treatment group who fail to comply<br>
λ2 = drop-in rate = proportion of participants in the placebo group who receive the active treatment outside the study protocol

In [3]:
# From equation 10.17 in the book...
gamma1 = 0.05 # drop-in rate of placebo group
gamma2 = 0.1 # drop-out rate of active treatment group
n_non_compliance = n / (1 - gamma1 - gamma2) ** 2 # we can use the approximation since both gamma1 and gamma2 are <= 0.1
print(f'Number of subjects in the study: {round(2*n_non_compliance)}')

Number of subjects in the study: 51021


The number of subjects increases significantly if we factor in drop-in and drop-outs. 

10.3 Answer Problem 10.1 if a one-sided test with power = .8 is used and compliance is perfect.

In [4]:
power = 0.8
alpha = 0.05
ratio = 1
nobs1 = None
n = sms.NormalIndPower().solve_power(es, nobs1, alpha, power, ratio=1, alternative='larger')
n = round(n)
print(round(n))
print(f'Number of subjects in the study: {round(2*n)}')

10845
Number of subjects in the study: 21690


Not surprisingly, fewer participants are required when the power requirement is decreased.

10.4 Suppose 11,000 men are actually enrolled in each treatment group. What would be the power of such a study if a two-sided test with α = .05 were used and compliance is perfect?

In [5]:
power = None
alpha = 0.05
ratio = 1
nobs1 = 11000
power = sms.NormalIndPower().solve_power(es, nobs1, alpha, power, ratio=1, alternative='larger')
print(f'Power of the study: {power}')

Power of the study: 0.8049190217750068


The power would be 80.4% chance of detecting a significant difference.

10.5 Answer Problem 10.4 if compliance is as given in Example 10.37.

*The power formula for the comparison of binomial proportions in Equation
10.14 also assumes perfect compliance. To correct these estimates for noncompli-
ance in a clinical trial setting, replace p , p , ∆, p , and q in Equation 10.14 with 12
p*, p* , ∆*, p*, q* as given in Equation 10.17.*

$$
\begin{aligned}
p_{1}^{*} &=\left(1-\lambda_{1}\right) p_{1}+\lambda_{1} p_{2} \\
p_{2}^{*} &=\left(1-\lambda_{2}\right) p_{2}+\lambda_{2} p_{1} \\
\end{aligned}
$$

In [6]:
nobs = 11000
alpha = 0.05
power = None

# adjust rates for dropin/dropout
p1_star = (1-gamma1)*p1 + gamma1*p2 # p1 = placebo rate
p2_star = (1-gamma2)*p2 + gamma2*p1 # p2 = aspirin rate
es = sms.proportion_effectsize(prop1=p1_star, prop2=p2_star, method='normal')
power = sms.NormalIndPower().solve_power(es, nobs1, alpha, power, ratio=1, alternative='two-sided')
print(power)

0.5643322663207736


The power drops to 56.4% when we incorporate the dropin and dropout rates.

10.6 Refer to Table 2.13 (p. 36).<br>
What significance test can be used to assess whether there is a relationship between receiving an antibiotic and receiving a bacterial culture while in the hospital?

A test of association / independence can be used. 

10.7 Perform the test in Problem 10.6, and report a p-value.

In [7]:
from scipy.stats import chi2_contingency
observed = [[7, 18], [6, 19]]
chi2, pvalue, dof, exp = chi2_contingency(observed, correction=True)
print(pvalue)

1.0


Since alpha > 0.05, we accept the null hypothesis that p1 = p2 and conclude that there is an association between receiving a bacterial culture and receiving an anti-biotitic.

**Gastroenterology**<br>
Two drugs (A, B) are compared for the medical treatment of duodenal ulcer. For this purpose, patients are carefully matched with regard to age, gender, and clinical condition. The treatment results based on 200 matched pairs show that for 89 matched pairs both treatments are effective; for 90 matched pairs both treatments are ineffective; for 5 matched pairs drug A is effective, whereas drug B is ineffective; and for 16 matched pairs drug B is effective, whereas drug A is ineffective.

10.8 What test procedure can be used to assess the results?

In [8]:
n_d = 5 + 16 # number of discordant pairs
print(f'Number of discordant pairs = {n_d}')

Number of discordant pairs = 21


Since the # of discordant pairs is > 20, and the data are paired involving proportions, McNemar's test can be used. 

10.9 Perform the test in Problem 10.8, and report a p-value.

In [9]:
from statsmodels.stats.contingency_tables import mcnemar
table = [[89,16],[5,90]]
results = mcnemar(table, exact=True, correction=True)
print(results)

pvalue      0.026603698730468753
statistic   5.0


Since p = 0.026 < 0.05, we can conclude that the drug does not have a significant affect on the treatment of duodenal ulcers.

**Sexually Transmitted Disease**<br>
Suppose researchers do an epidemiologic investigation of people entering a sexually transmitted disease clinic. They find that 160 of 200 patients who are diagnosed as having gonorrhea and 50 of 105 patients who are diagnosed as having nongonococcal urethritis have had previous episodes of urethritis.

10.13 Are the present diagnosis and prior episodes of urethritis associated?

So, if urethritis is unrelated to diagnoses of gonorrhea and nongonococcal urethritis, then the proportion of patients with gonorrhea who have had previous episodes of urethritis (p1) should be 0.5 (or at least, not significantly different than 0.5). The same goes for the proportion patients diagnosed with nongonococcal urethritis (p2). Therefore, we can use an 2 x K contingency table and the chi-square test for association to test whether an association exists or not, (assuming the assumptions for the test are met). 

So therefore:<br>
H0: There is no association, i.e. p1 = p2 = p3 (= 0.5)<br>
HA: There is an association, i.e. at least two samples are not from the same population (don't have a rate of 0.5), which would mean that there is no significant association. 

**Assumptions:**
- No more than 1/5 of the cells have expected values < 5. and
- No cell has an expected value < 1.

The question is talking about two groups (i.e. samples) of patients - those with gonorrhea, and patients without gonorrhea. The question gives us the data on how many patients within each of the two groups had previous diagnoses of urethritis, and then asks if having been diagnosed with urethritis in the past is associated with a diagnosis of gonorrhea.

In [108]:
## solution based on solution from chegg, now that I understand what the question is asking...
from scipy.stats import chi2_contingency
observed = [[160, 40], [50, 55]]
chi2, pvalue, dof, exp = chi2_contingency(observed)
print(chi2, pvalue)

32.170242645303745 1.4123745276875494e-08


- H0: p1 = p2
- HA: p1 != p2

Since p < 0.05, we can reject the null and accept HA, that there is an association between previous diagnoses of urehtritis and gonnohrea.

In [24]:
p1 = 160/200
print(p1)
p2 = 50/105
print(p2)
n_total = 305
p3 = 0.5

0.8
0.47619047619047616


In [19]:
305 * 0.5

152.5

In [25]:
from scipy.stats import chi2_contingency
observed = [[40, 55, 152.5], [160, 50, 152.5]]
chi2, pvalue, dof, exp = chi2_contingency(observed)
print(pd.DataFrame(exp))
print(pvalue)

            0          1       2
0   81.147541  42.602459  123.75
1  118.852459  62.397541  181.25
4.139514165505269e-12


We can reject H0 and accept HA, that there is an association between urethritis and at least one of the populations under study (gonorrhea and nongonococcal urethritis).

But which one would it be?

In [35]:
from scipy.stats import chi2_contingency
observed = [[40, 152.5], [160, 152.5]]
chi2, pvalue, dof, exp = chi2_contingency(observed)
print(pd.DataFrame(exp))
print(pvalue)

            0           1
0   76.237624  116.262376
1  123.762376  188.737624
2.1551565213831957e-11


alternatively, using a z-test:<br>
H0: p1 = p3<br>
Ha: p1 != p3<br>

In [37]:
from statsmodels.stats.weightstats import ztest
x1 = np.concatenate((np.ones(160), np.zeros(40)))
tstat, pvalue = ztest(x1=x1, x2=None, value=0.5, alternative='two-sided')
print(pvalue)

3.687535748815112e-26


And we can reject the null hypothesis and accept the alternative, that the rate of urethritis for patients diagnosed with gonnorhea is not 0.5, and thus there is some association for the two groups. 

What about the group diagnosed with nongonococcal urethritis?

In [39]:
from statsmodels.stats.weightstats import ztest
x1 = np.concatenate((np.ones(50), np.zeros(55)))
tstat, pvalue = ztest(x1=x1, x2=None, value=0.5, alternative='two-sided')
print(pvalue)

0.6268449135173175


Since p > 0.05, we cannot reject the null and must accept it - there is no association between diagnoses of nongonococcal urethritis and prior diagnoses of urethritis. 

10.14<br>

A 1980 study investigated the relationship between the use of OCs and the development of endometrial cancer [9]. The researchers found that of 117 endometrial-cancer patients, 6 had used the OC Oracon at some time in their lives, whereas 8 of the 395 controls had used this agent. Test for an association between the use of Oracon and the incidence of endometrial cancer, using a two-tailed test.

In [10]:
p1 = 6 / 117
p2 = 8 / 395

H0: p1 = p2<br>
Ha: p1 != p2

In [11]:
count = [6, 8]
nobs = [117, 395]

In [12]:
# z-test method
# from statsmodels.stats.weightstats import ztest
# ztest(x1, x2=None, value=0, alternative='two-sided', usevar='pooled', ddof=1.0)

from statsmodels.stats.proportion import proportions_ztest
proportions_ztest(count, nobs, value=None, alternative='two-sided')

# from statsmodels.stats.proportion import test_proportions_2indep
# test_proportions_2indep(count1, nobs1, count2, nobs2, value=None, method=None, compare='diff', alternative='two-sided', correction=True, return_results=True

(1.8076484117100218, 0.07066123842631354)

In [13]:
from scipy.stats import chi2_contingency
observed = [[111, 6], [387, 8]]
chi2, pvalue, dof, exp = chi2_contingency(observed, correction=False)
print(chi2, pvalue)

3.2675927803577642 0.07066123842631342


In [14]:
print(exp)

[[113.80078125   3.19921875]
 [384.19921875  10.80078125]]


since p = 0.07 > 0.05, we cannt reject the null that p1 = p2, and so we can conclude that the rate of cancer in the two groups is not significantly different. 

10.19 

Provide a point estimate and a 95% confidence interval for the prevalence of otorrhea at 2 weeks in the observation group.

In [15]:
from statsmodels.stats.proporxtktion import proportion_confint
count = 41
nobs = 75
ci = proportion_confint(count, nobs, alpha=0.05, method='normal')
p = count / nobs
print(f'p = {p}')
print(f'ci = {ci}')

p = 0.5466666666666666
ci = (0.4340020397707578, 0.6593312935625755)


10.23<br>

Test whether the distribution of stage of disease is significantly different between Caucasian and African American women with breast cancer who are younger than 50 years of age. Please provide a p-value (two-tailed). Ignore the unstaged cases in your analysis.

Data are available for stage of disease at diagnosis for women with breast cancer by age and race as shown in Table 10.23

Variables: 
- race (caucasian, african american)
- stage of breast cancer (localized, regional, distant)<br>

Measure: proportions

Method: chi-square goodness of fit test. The data can be cast into a 2 x 3 contingency table, and a chi-square test statistic can be calculated.

Hypotheses:
- H0: Data come from the same distribution
- Ha: Data do not come from the same distribution

In [173]:
n1 = 53060
n2 = 8063
p1 = [54, 41, 3]
p2 = [46, 46, 7]
group1 = np.multiply(n1, np.divide(p1, 100))
group2 = np.multiply(n2, np.divide(p2, 100))

In [171]:
# from scipy.stats import chisquare
# chisq, pvalue = chisquare(f_obs=p1, f_exp=p2)
# print(chisq, pvalue)

In [172]:
from scipy.stats import chi2_contingency
chisq, pvalue, dof, exp = chi2_contingency(np.stack([p1,p2]), correction=False)
print(chisq, pvalue)

2.52234517352744 0.2833216124730528


In [178]:
from scipy.stats import chi2_contingency
chisq, pvalue, dof, exp = chi2_contingency(np.round(np.stack([group1,group2])), correction=True)
print(chisq, pvalue)

439.23059897313317 4.19070694165862e-96


p = 0.28 > 0.05, so we accept the null hypothesis and establish that the two groups do have the same underlying distribution (of the stages of disease for caucasian vs african american women with breast cancer who are younger than 50 years old).

10.24 Test whether the 5-year survival rate for breast can- cer is significantly different between African American and Caucasian women who are younger than 50 years of age and have localized disease. Provide a p-value (two-tailed).

The 5-year survival rates by stage of disease, age at diagno- sis, and race are provided in Table 10.24.

In [92]:
p1 = 96.5/100
p2 = 91.6/100
n1 = 53060
n2 = 8063
count = [p1 * n1, p2 * n2]
nobs = [n1, n2]

# from statsmodels.stats.weightstats import ztest
# ztest(p1, p2)

# from statsmodels.stats.proportion import proportions_ztest
proportions_ztest(count, nobs, value=None, alternative='two-sided', prop_var=False)

(20.56301911717198, 5.88545679070323e-94)

10.25 Is there any relationship between the type of treatment and the response? What form does the relationship take?

variables:
- treatment
- response

measurement:
- count (of persons)

hypotheses:
- H0: There is no association between treatment and outcomes/response
- Ha: There is an association between treatment and the outcome/response

method:
- chi-square test for association, with a 3x3 contingency table

In [107]:
f_obs = [[40, 30, 130], [10, 20, 70], [15, 40, 45]]
print('f_obs:')
print(pd.DataFrame(f_obs))
from scipy.stats import chi2_contingency
chisq, pvalue, dof, exp = chi2_contingency(f_obs, correction=False)
print(f'dof={dof}')
print('f_exp:')
print(pd.DataFrame(exp))
print(f'chisq={chisq}')
print(f'pvalue={pvalue}')

f_obs:
    0   1    2
0  40  30  130
1  10  20   70
2  15  40   45
dof=4
f_exp:
       0     1       2
0  32.50  45.0  122.50
1  16.25  22.5   61.25
2  16.25  22.5   61.25
chisq=29.140066282923428
pvalue=7.3215754548823335e-06


p << 0.05, so we can reject the null and accept the alternative that there is an association between treatment and outcomes.

10.26 Suppose either a positive smear or a positive culture is regarded as a positive response and distinguished from the negative smear, negative culture response. Is there an as- sociation between the type of treatment and this measure of response?

In [108]:
f_obs = [[40 + 30, 130], [10 + 20, 70], [15 + 40, 45]]
from scipy.stats import chi2_contingency
chisq, pvalue, dof, exp = chi2_contingency(f_obs, correction=False)
print(dof)
print(exp)
print(chisq)
print(pvalue)

2
[[ 77.5  122.5 ]
 [ 38.75  61.25]
 [ 38.75  61.25]]
15.53653719552337
0.0004229449210153579


p = 0.0004 < 0.05, so there is an association.

10.32<br>
Suppose all the matched pairs in Table10.27 are considered. What method of analysis can be used to test whether there is an association between widowhood and mortality?

Hypotheses:
- H0: No association between widowhood and mortality; i.e Pr(discordant pair is type A) = 1/2
- Ha: There is an association

Test statistic:
- X2 =(nA −nB −1) / (nA +nB)

Method:
- McNemar's test for correlated proportions

Assumptions:
- n<sub>D</sub> (# discordant pairs) >= 20
- Data are paired

since n<sub>D</sub> = 144 + 103 = 247, the requirement for Mcnemar's test is satisfied.

10.33 Implement the test in Problem 10.32, and report a p-value.

$$X^{2}=\left(\left|n_{A}-n_{B}\right|-1\right)^{2} /\left(n_{A}+n_{B}\right)$$

In [135]:
chisq = (np.abs(144 - 103) - 1)**2 / (144 + 103)
print(chisq)
import scipy
rv = scipy.stats.chi2(1)
pvalue = 1 - rv.cdf(chisq)
print(pvalue)

6.477732793522267
0.010923422995323562


Since p < 0.05, we accept the alternative hypothesis that there is an association.

10.34 

Answer the same question as Problem 10.32 con- sidering 36- to 45-year-old males only.

In [136]:
na = 4 + 3
nb = 8 + 2
nd = na + nb
print(nd)

17


Since n<sub>D</sub> is < 20, we must use the exact method.

10.35 Implement the test in Problem 10.34, and report a p-value.

10.36 

Based on all matched pairs, how much power did the study just mentioned have vs. the alternative hypothesis that a widower is twice as likely to die before a married person of the same age and gender, assuming that all age groups are considered?

Formula for **Power Achieved in Comparing Two Binomial Proportions Using a Two-Sided Test
with Significance Level α (Paired-Sample Case)**<br>
$$
Power =\Phi\left[\frac{1}{2 \sqrt{p_{A} q_{A}}}\left(z_{\alpha / 2}+2\left|p_{A}-.5\right| \sqrt{n p_{D}}\right)\right]
$$

In [157]:
n = 151 + 544 # widowers + widows
p_d = (103 + 144) / n
p_a = 2/3
q_a = 1 - p_a

value = (1/(2*np.sqrt(p_a*q_a))) * (-1.96 + 2 * np.abs(p_a - 0.5) * np.sqrt(n * p_d))
rv = scipy.stats.norm(loc=0, scale=1)
rv.cdf(value)

0.9997470697037882

Power is basically 100%.

Cardiovascular Disease

A secondary prevention trial of lipid lowering is planned in patients with previous myocardial infarction (MI). Patients are to be randomized to either a treatment group receiving diet therapy and cholesterol-lowering drugs or a control group receiving diet therapy and placebo pills. The study endpoint is to be a combined endpoint consisting of either definite fatal coronary heart disease (CHD) or nonfatal MI (i.e., a new nonfatal MI distinct from previous events). Sup- pose it is projected that the incidence of combined events among controls is 7% per year.


10.42

What proportion of controls will have events over 5 years? Hint: Assume no deaths due to non-CHD causes.

In [158]:
1 - (1 - 0.07)**5

0.3043116307000002

30.4% of controls will have experienced either fatal CHD or MI.

10.43 

Suppose the treatment benefit is projected to be a reduction in the 5-year event rate by 30%. What is the expected event rate in the treated group?

I assume that the decrease is a decrease in *relative* risk, which would mean...

In [161]:
0.304 * (1-0.3)

0.2128

Treatment group has a 5 year proportion of events calculated as 21.2%

## Review Questions 10E

1 (a) What is the difference between the chi-square test for trend and the chi-square test for heterogeneity?

test for heterogeneity just checks to see if the proportions in each group are all equal or not (basically, whether the first variable is related to the second), whereas the chi-square test for trend checks to see if there is a *specific, trending* relationship between the two variables (treatment/outcome).

(b) When do we use each test?

Test for heterogeneity: when you want to know if two variables are related (like taking a certain drug and survival rates)

Test for trend: 